In [79]:
import argparse
import os
import model_utils
import pickle
import torch
import numpy as np
import scipy

from torch.autograd import Variable

import pdb

parent_dir = '/root/projects/pytorch_integrated_cell/test_aaegan/aaegan3Dv6_128D_latent_v4'
gpu_ids = [0, 1]
batch_size = 100

# parser = argparse.ArgumentParser()
# parser.add_argument('--parent_dir', help='save dir')
# parser.add_argument('--gpu_ids', nargs='+', type=int, default=0, help='gpu id')
# parser.add_argument('--batch_size', type=int, default=400, help='batch_size')
# parser.add_argument('--overwrite', type=bool, default=False, help='overwrite existing results')
# args = parser.parse_args()

# model_dir = args.parent_dir + os.sep + 'struct_model' 


figure_dir = parent_dir + os.sep + 'figures'

if not os.path.exists(figure_dir):
    os.makedirs(figure_dir)



torch.manual_seed(2)
torch.cuda.manual_seed(2)
np.random.seed(2)




model_dir = parent_dir + os.sep + 'ref_model'
opt_ref = pickle.load(open( '{0}/opt.pkl'.format(model_dir), "rb" ))
print(opt_ref)
opt_ref.gpu_ids = gpu_ids

opt_ref.save_dir = model_dir
models, _, _, logger_ref, _ = model_utils.load_model(opt_ref.model_name, opt_ref)
enc_ref = models['enc']
dec_ref = models['dec']

enc_ref.train(False)
dec_ref.train(False)
models = None

dp_ref = model_utils.load_data_provider(opt_ref.data_save_path, opt_ref.imdir, opt_ref.dataProvider)
dp_ref.opts['channelInds'] = opt_ref.channelInds

model_dir = parent_dir + os.sep + 'struct_model'
opt = pickle.load(open( '{0}/opt.pkl'.format(model_dir), "rb" ))
print(opt)
opt.gpu_ids = gpu_ids

opt.save_dir = model_dir
models, _, _, logger, _ = model_utils.load_model(opt.model_name, opt)
enc = models['enc']
dec = models['dec']

enc.train(False)
dec.train(False)
models = None

dp = model_utils.load_data_provider(opt.data_save_path, opt.imdir, opt.dataProvider)


print('done loading models')

Namespace(Diters=5, DitersAlt=100, batch_size=30, channelInds=[0, 2], channels_pt1=[0, 2], channels_pt2=[0, 1, 2], critRecon='BCELoss', dataProvider='DataProvider3Dh5', data_save_path='./test_aaegan/aaegan3Dv6_128D_latent_v4/data.pyt', decDRatio=0.0001, dtype='float', encDRatio=0.0001, gpu_ids=[0, 1], imdir='/root/results/ipp_dataset_cellnuc_seg_curated_8_24_17', latentDistribution='gaussian', lrDec=5e-05, lrDecD=5e-05, lrEnc=5e-05, lrEncD=0.01, model_name='aaegan3Dv6', myseed=0, nClasses=0, nRef=0, nch=2, ndat=14302, nepochs=250, nepochs_pt2=300, nlatentdim=128, noise=1e-05, optimizer='adam', saveProgressIter=1, saveStateIter=1, save_dir='./test_aaegan/aaegan3Dv6_128D_latent_v4//ref_model', save_parent='./test_aaegan/aaegan3Dv6_128D_latent_v4/', train_module='aaegan_trainv3')
Loading from /root/projects/pytorch_integrated_cell/test_aaegan/aaegan3Dv6_128D_latent_v4/ref_model
Namespace(Diters=5, DitersAlt=100, batch_size=30, channelInds=[0, 1, 2], channels_pt1=[0, 2], channels_pt2=[0, 1

In [ ]:
parent_dir + os.sep + 'ref_model'

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from imgToProjection import imgtoprojection
from tqdm import tqdm
from sklearn.metrics import confusion_matrix
    
def tensor2img(imgs, colors=None):

    
    if colors is None:
            colormap = 'hsv'
            colors = plt.get_cmap(colormap)(np.linspace(0, 1, imgs.shape[1]+1))
    
    imgs = imgs.numpy()
    im_out = list()
    
    #for every image
    for i in range(0, imgs.shape[0]):
        img = imgs[i]
        
        #contrast adjust each channel
        for j in range(0, img.shape[0]):
            maxval = np.max(img[j])
            if maxval > 0:
                img[j] = img[j]/maxval
            
        
        img = imgtoprojection(np.swapaxes(img, 1, 3), proj_all=True, colors = colors, global_adjust=True)
        img = np.swapaxes(img, 0, 2)
        
        img = np.flip(np.flip(img,0), 1)
        
        #very light border between each projection
        img[:,-2:,:] = .25
        img[-2:,:,:] = .25
        
        im_out.append(img)

    img = np.concatenate(im_out, 1)

    return img

def print_fig1(dp, save_path, save_path_struct, n_per_row = 6, colors=[[1,0,1], [1,1,0], [0,1,1]]):
    #this figure is #classes by n_per_row max project image of cells. 

    ntrain = dp.get_n_dat('train')
    
    train_classes = dp.get_classes(np.arange(0, ntrain), 'train').numpy()
    class_names = dp.label_names
    
    image_list = list()
    image_list_struct = list()
    
    for i in range(0, len(class_names)):
        inds = np.where(train_classes == i)[0]
        np.random.shuffle(inds)
        
        #make the CMY image
        images = dp.get_images(inds[0:n_per_row], 'train')
        images_flat = tensor2img(images, colors)
        image_list.append(images_flat)
        
        #make the struct-only Yellow image
        images[:, [0,2], :, :,:] = images[:, [0,2], :, :,:].fill_(0) #not sure why i have to do it this way, but it doesn't work without the assigment
        
        images_flat_struct = tensor2img(images, colors)
        image_list_struct.append(images_flat_struct)
        
        
    images = np.concatenate(image_list, axis=0)
    scipy.misc.imsave(save_path, images)
    
    images_struct = np.concatenate(image_list_struct, axis=0)
    scipy.misc.imsave(save_path_struct, images_struct)
                                   
def print_fig2(enc, dec, dp, save_path, save_path_struct, n_per_row = 6, colors=[[1,0,1], [1,1,0], [0,1,1]], required_test_inds=None):
    ntrain = dp.get_n_dat('train')
    train_classes = dp.get_classes(np.arange(0, ntrain), 'train').numpy()
    
    class_names = dp.label_names
    
    nclasses = len(class_names)
    train_inds = np.int_(np.zeros(nclasses))
    for i in range(0, len(class_names)):
        images_w = list()
        class_inds = np.where(train_classes == i)[0]
        np.random.shuffle(class_inds)
        train_inds[i] = class_inds[0]
        
    train_images = dp.get_images(train_inds, 'train')
    train_images_struct = train_images.clone()
    train_images_struct[:, [0,2], :, :,:] = train_images_struct[:, [0,2], :, :,:].fill_(0)
    
    
    n_per_row_tmp = n_per_row - len(required_test_inds)
    
    ntest = dp.get_n_dat('test')
    test_inds= np.arange(0, ntest)
    np.random.shuffle(test_inds)
    test_inds = test_inds[np.arange(0, n_per_row_tmp-1)]
    
    test_inds = np.concatenate([test_inds, required_test_inds])
    
    test_images = dp.get_images(test_inds, 'test')
    
    images_h = list()
    images_h_struct = list()   
    
    for i in range(0, len(class_names)):
        images_w = list()
        images_w_struct = list()
        
        train_inds = np.where(train_classes == i)[0]
        np.random.shuffle(train_inds)
        
        for j in range(0, n_per_row-1):
            image = Variable(torch.unsqueeze(test_images[j], 0))
            z = enc(image)
            
            z[0].data.fill_(-100)
            z[0].data[0,i] = 0

            z[-1].data.fill_(0)
    
            image = dec(z)
        
            image = image.data.cpu()
        
            image[:, [0,2], :, :,:] = test_images[j][[0,2],:,:,:]
        
            images_w.append(tensor2img(image, colors))
            
            image[:, [0,2], :, :,:] = image[:, [0,2], :, :,:].fill_(0)
            images_w_struct.append(tensor2img(image, colors))
        
        #CMY version
        images_w = np.concatenate(images_w, axis=1)
        images_w[:,-2:,:] = 0.5
        image_flat = tensor2img(torch.unsqueeze(train_images[i], 0), colors)
        images_w = np.concatenate([images_w, image_flat], axis=1)        
        images_h.append(images_w)
        
        #Struct-only Yellow version
        images_w_struct = np.concatenate(images_w_struct, axis=1)
        images_w_struct[:,-2:,:] = 0.5
        image_flat = tensor2img(torch.unsqueeze(train_images_struct[i], 0), colors)
        images_w_struct = np.concatenate([images_w_struct, image_flat], axis=1)        
        images_h_struct.append(images_w_struct)
        
    images = np.concatenate(images_h, axis=0)
    scipy.misc.imsave(save_path, images)
    
    images_struct = np.concatenate(images_h_struct, axis=0)
    scipy.misc.imsave(save_path_struct, images_struct)
    
def print_conf_mat(enc, dp, figure_dir, batch_size, default_gpu_id):


    train_or_test_groups = ['test', 'train']

    cf_out = dict()

    for train_or_test in train_or_test_groups:
        save_path = figure_dir + os.sep + 'conf_mat_' + train_or_test + '.csv'
        
        
        y_hat = list()
        y = list()

        ndat = dp.get_n_dat(train_or_test)

        intervals = np.arange(0, ndat, batch_size)

        for i in tqdm(intervals):
            batch_range = i + batch_size
            if batch_range > ndat:
                batch_range = ndat


            dat_inds = np.arange(i, batch_range)

            images = Variable(dp.get_images(dat_inds,train_or_test)).cuda(default_gpu_id)
            labels = dp.get_classes(dat_inds,train_or_test).numpy()

            z = enc(images)

            y_hat.append(np.argmax(z[0].data.cpu().numpy(), axis=1))
            y.append(labels)

            del images, labels, z


        y = np.concatenate(y, axis=0)
        y_hat = np.concatenate(y_hat, axis=0)

        cf = confusion_matrix(y, y_hat)
        pd.DataFrame(cf, columns=dp.label_names).to_csv(save_path, index=False)

        cf_out[train_or_test] = cf 
        
    return cf_out

def print_recon(enc, dec, dp, save_path, default_gpu_id, colors=[[1,0,1], [1,1,0], [0,1,1]]):
    
    train_or_test_groups = ['train', 'test'] 

    class_names = dp.label_names
    

    images_train_test = list()
    
    for train_or_test in train_or_test_groups:
        
        ntrain = dp.get_n_dat(train_or_test)
        train_classes = dp.get_classes(np.arange(0, ntrain), train_or_test).numpy()
        
        image_in = list()
        image_out = list()
        
        for i in range(0, len(class_names)):
            inds = np.where(train_classes == i)[0]
#             np.random.shuffle(inds)

            #make the CMY image
            images = dp.get_images([inds[0]], train_or_test)

            image_in_flat = tensor2img(images, colors)
            image_in.append(image_in_flat)
            
            image_recon = dec(enc(Variable(images).cuda(default_gpu_id)))
            image_out_flat = tensor2img(image_recon.data.cpu(), colors)
            image_out.append(image_out_flat)

#             #make the struct-only Yellow image
#             images[:, [0,2], :, :,:] = images[:, [0,2], :, :,:].fill_(0) #not sure why i have to do it this way, but it doesn't work without the assigment

#             images_flat_struct = tensor2img(images, colors)
#             image_list_struct.append(images_flat_struct)
        
        
        image_in = np.concatenate(image_in, axis=1)
        image_out = np.concatenate(image_out, axis=1)
        
        images = np.concatenate([image_in, image_out], axis=0)
        images_train_test.append(images)
        
    images_train_test = np.concatenate(images_train_test, axis=0)
    scipy.misc.imsave(save_path, images_train_test)
    
#     images_struct = np.concatenate(image_list_struct, axis=0)
#     scipy.misc.imsave(save_path_struct, images_struct)


def print_history(logger, save_path, r_or_s):
    field_order = ['reconLoss',
#                      'minimaxEncDLoss',
                     'encDLoss',
#                      'minimaxDecDLoss',
                     'decDLoss',
                     'classLoss',
                     'refLoss']
    
    
    field_dict = {'reconLoss': '$\mathcal{L}_{X_{' + r_or_s + '}}$',
                  'minimaxEncDLoss': '$\mathcal{L}_{mmEncD^{' + r_or_s + '}}$',
                  'encDLoss': '$\mathcal{L}_{EncD^{' + r_or_s + '}}$',
                  'minimaxDecDLoss': '$\mathcal{L}_{mmDecD^{' + r_or_s + '}}$',
                  'decDLoss': '$\mathcal{L}_{DecD^{' + r_or_s + '}}$',
                  'classLoss': '$\mathcal{L}_{\hat{' + r_or_s + '}}$',
                  'refLoss': '$\mathcal{L}_{\hat{x}^{' + r_or_s + '}}$'}
    
    
    plt.figure()

    for i in range(0, len(field_order)):
        field = field_order[i]
        if field in logger.fields:
            plt.plot(logger.log['iter'], logger.log[field], label=field_dict[field])

    plt.legend()
    plt.title('History')
    plt.xlabel('iteration')
    plt.ylabel('loss')
    plt.savefig(save_path, bbox_inches='tight')
    plt.close()
    
def print_latent_plot(embeddings, save_path, r_or_s, dims=[0,1]):
    
    train_or_test_groups = ['train', 'test'] 
    
    plt.figure(num=None, figsize=(4, 4), dpi=120, facecolor='w', edgecolor='k')
    
    handles = list()
    for train_or_test in train_or_test_groups:
        embed_points = embeddings[train_or_test].numpy()
        h = plt.scatter(embed_points[:,dims[0]], embed_points[:,dims[1]], s=1, alpha=0.5)
        
        h = mpatches.Patch(color=h.get_facecolor()[0], label=train_or_test)
        handles.append(h)
        
    plt.xlabel('$z^{' + r_or_s + '}_{' + str(dims[0]+1) + '}$')
    plt.ylabel('$z^{' + r_or_s + '}_{' + str(dims[1]+1) + '}$')    
            
    plt.legend(handles, train_or_test_groups, loc=1)
        
    ax_range = 4
#     plt.axis([-ax_range, ax_range, -ax_range, ax_range])
#     plt.axis('equal')
    
    plt.xlim(-ax_range, ax_range)
    plt.ylim(-ax_range, ax_range)
    plt.gca().set_aspect('equal', adjustable='box')


    plt.savefig(save_path, bbox_inches='tight')
    
    plt.close('all')
    
    pass
    
def print_latent_grid(dec, ref_or_class_num, save_path, default_gpu_id, dims=[0,1], std=1.5, nsamples=5, colors=[[1,0,1], [1,1,0], [0,1,1]]):
    
    z = list()
    
    if ref_or_class_num is not 'ref':
        label = Variable(torch.Tensor(1, 10).fill_(-50).cuda(default_gpu_id))
        label.data[0][ref_or_class_num] = 0

        z.append(label)
        
        z.append(Variable(torch.Tensor(1, 128).fill_(0).cuda(default_gpu_id)))

        
    
    
    z.append(Variable(torch.Tensor(1, 128).fill_(0).cuda(default_gpu_id)))
    
    samples = np.linspace(-std, std, nsamples)
    
    images_w = list()
    for i in samples:
        images_h = list()
        for j in samples:
            z[-1].data[0][dims[0]] = i
            z[-1].data[0][dims[1]] = j
    
            im_out = dec(z)
            image_out_flat = tensor2img(im_out.data.cpu(), colors)
            images_h.append(image_out_flat)
            
        images_w.append(np.concatenate(images_h, axis=0))
        
    image_out = np.concatenate(images_w, axis=1)
    
    scipy.misc.imsave(save_path, image_out)
    

In [166]:
fig1_path = figure_dir + os.sep + 'fig1.png'
fig1_s1_path = figure_dir + os.sep + 'fig1_s1.png'
print_fig1(dp, fig1_path, fig1_s1_path)


In [164]:
fig2_path = figure_dir + os.sep + 'fig2.png'
fig2_s2_path = figure_dir + os.sep + 'fig2_s2.png'
print_fig2(enc, dec, dp, fig2_path, fig2_s2_path, n_per_row=6, required_test_inds=[975])

In [ ]:
history_ref_path = figure_dir + os.sep + 'ref_history.png'
print_history(logger_ref, history_ref_path)

history_struct_path = figure_dir + os.sep + 'struct_history.png'
print_history(logger, history_struct_path)



In [239]:
recon_path = figure_dir + os.sep + 'struct_recon.png'
print_recon(enc, dec, dp, recon_path, gpu_ids[0])


recon_path = figure_dir + os.sep + 'ref_recon.png'
print_recon(enc_ref, dec_ref, dp_ref, recon_path, gpu_ids[0], colors=[[1,0,1], [0,1,1]])

In [ ]:
conf_mat_path = figure_dir + os.sep + 'conf_mat'
print_conf_mat(enc, dp, figure_dir, 200, gpu_ids[0])

In [238]:
latent_recon_path = figure_dir + os.sep + 'ref_latent.png'
embeddings_path = parent_dir + os.sep + 'ref_model' + os.sep + 'embeddings.pkl'
embeddings = model_utils.load_embeddings(embeddings_path)

print_latent_plot(embeddings, latent_recon_path, 'r')


latent_recon_path = figure_dir + os.sep + 'struct_latent.png'
embeddings_path = parent_dir + os.sep + 'struct_model' + os.sep + 'embeddings.pkl'
embeddings = model_utils.load_embeddings(embeddings_path)

print_latent_plot(embeddings, latent_recon_path, 's')


In [235]:
std = 3

latent_grid_path = figure_dir + os.sep + 'grid_ref.png'
print_latent_grid(dec_ref, 'ref', latent_grid_path, gpu_ids[0], std=std, colors=[[1,0,1], [0,1,1]])

latent_grid_path = figure_dir + os.sep + 'grid_struct.png'

for i in range(0, dp.get_n_classes()):
    latent_grid_path = figure_dir + os.sep + 'grid_struct_' + dp.label_names[i] + '.png'
    print_latent_grid(dec, i, latent_grid_path, gpu_ids[0], std=std, colors=[[0,0,0], [1,1,0], [0,0,0]])





In [ ]:


print(np.where(dp.data['test']['inds']==8824))

print(np.where(dp.data['test']['inds']==9238))

print(np.where(dp.data['test']['inds']==9538))

print(np.where(dp.data['test']['inds']==9695))

In [236]:
save_path = figure_dir + os.sep + 'tmp.png'
colors=[[1,0,1], [1,1,0], [0,1,1]]

images = Variable(dp.get_images([1], 'train')).cuda(gpu_ids[0])
z = enc(images)
z[1].data[0].fill_(0)
img_out = dec(z)
img_out = img_out.data.cpu()


scipy.misc.imsave(save_path, tensor2img(img_out, colors))

print(z)

[Variable containing:

Columns 0 to 7 
  0.0000 -24.6505 -23.4934 -35.1414 -23.4414 -34.4089 -25.4520 -27.4133

Columns 8 to 9 
-21.8785 -22.0951
[torch.cuda.FloatTensor of size 1x10 (GPU 0)]
, Variable containing:

Columns 0 to 12 
    0     0     0     0     0     0     0     0     0     0     0     0     0

Columns 13 to 25 
    0     0     0     0     0     0     0     0     0     0     0     0     0

Columns 26 to 38 
    0     0     0     0     0     0     0     0     0     0     0     0     0

Columns 39 to 51 
    0     0     0     0     0     0     0     0     0     0     0     0     0

Columns 52 to 64 
    0     0     0     0     0     0     0     0     0     0     0     0     0

Columns 65 to 77 
    0     0     0     0     0     0     0     0     0     0     0     0     0

Columns 78 to 90 
    0     0     0     0     0     0     0     0     0     0     0     0     0

Columns 91 to 103 
    0     0     0     0     0     0     0     0     0     0     0     0     0

Columns 

In [116]:
np.arange(0,128,2)

array([  0,   2,   4,   6,   8,  10,  12,  14,  16,  18,  20,  22,  24,
        26,  28,  30,  32,  34,  36,  38,  40,  42,  44,  46,  48,  50,
        52,  54,  56,  58,  60,  62,  64,  66,  68,  70,  72,  74,  76,
        78,  80,  82,  84,  86,  88,  90,  92,  94,  96,  98, 100, 102,
       104, 106, 108, 110, 112, 114, 116, 118, 120, 122, 124, 126])

In [26]:
dp.get_n_dat('train')+dp.get_n_dat('test')

15901

In [137]:
dp.label_names

array(['Alpha actinin', 'Alpha tubulin', 'Beta actin', 'Desmoplakin',
       'Fibrillarin', 'Lamin B1', 'Myosin IIB', 'Sec61 beta', 'Tom20',
       'ZO1'],
      dtype='<U13')